# Linearity um Saatpunkte
Dieses Notebook erzeugt zwei Visualisierungen. Die erste öffnet sich in einem separaten Fenster und zeigt die Kandidatenpunkte (niedrige/hohe Intensität blau bzw. grau) und die Saatpunkte (rot). Die zweite wird im Notebook selbst angezeigt und zeigt die linearity in der Nachbarschaft der Saatpunkte sowie die Richtung des ersten Eigenvektors.

In [ ]:
import pdal 
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import pyvista as pv

from scipy.spatial import KDTree
import os
import json
import time 

from interessant import * # Bei Änderungen Kernel neu starten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
run = run24
#run = run14


# Bahnsteig: 29; Gleis hohe Intensität: 11; Weiche B: 16; Unterirdischer Bhf: 20; Gleis weit abseits: 23; Betondeckel: 28; Zug run 14 A (in run24 Achszähler): 6; 
# Viele Gleise: 33; Anfang Weiche: 34; OLA gleiche H: 35; Y: 37
key = list(interessant.keys())[38] 
filename = interessant[key] # filename = interessant['OLA gleiche Höhe wie Gleis']
print(key, filename)

filename = os.path.join(run, filename)
if not os.path.exists(filename):
    raise FileNotFoundError(filename)

Weiche C 4473850_5336225.copc.laz


## Parameter

In [3]:
thresh = 8  # z.B. 5 oder 8
majority_tresh  = 0.5 # Erster Durchgang 0.3, bei "Gleis hohe Intensität" gibt 0.5 ein viel besseres Ergebnis

voxel_size = 1.0

voxel_size = 25 / 30
print("Voxel size:", voxel_size)

minimum_points = 50 # Erste Versuche mit 100, aber viel schwarz bei abseits liegenden Gleisen. 50 ist besser.

intensity_threshold = 14500
downsample_radius = 0.3
neighborhood_radius = 0.5

Voxel size: 0.8333333333333334


## Voxelfilter

In [4]:
start = time.time()

pipeline = pdal.Pipeline([pdal.Reader(filename)])
pipeline.execute()
points = pipeline.arrays[0]

In [5]:
xyz = np.vstack((points['X'], points['Y'], points['Z'])).transpose()

In [6]:
points['Classification'] = 0 # Unclassified
RAIL = 20

In [7]:
maxp = xyz.max(axis=0)
minp = xyz.min(axis=0)

In [8]:
voxels = xyz.copy()
voxels[:, :2] = ((xyz[:, :2] - minp[:2]) // voxel_size).astype(int)

In [9]:
# Anzahl der Voxel checken
np.ceil((maxp[:2] - minp[:2]) / voxel_size).astype(int)

array([30, 30])

In [10]:
from collections import defaultdict
voxel_dict = defaultdict(list)
index_dict = defaultdict(list)

# Füllen des Dictionaries
for idx, (point, voxel) in enumerate(zip(xyz, voxels)):
    voxel_key = tuple(voxel[:2])
    voxel_dict[voxel_key].append(point[2])
    index_dict[voxel_key].append(idx)

In [11]:
for key, z_values in voxel_dict.items():
    
    # Threshold on number of points in voxel
    if len(z_values) < minimum_points:
        continue

    indices = np.array(index_dict[key])
    z_values = np.array(z_values)
    ground_level = np.percentile(z_values, 10) # 10% Percentile
    # Check that there are almost no points 0.5 to 4.5 m above the ground
    # But allow for some noise
    # thresh = 3 # Der einfachheit halber oben
    count = ((z_values > ground_level + 0.5) & (z_values < ground_level + 4.5)).sum()

    if count <= thresh:
        # Look for points within 0.5 m above ground and get 98% percentile ODER 99.5
        mask = (z_values > ground_level) & (z_values < ground_level + 0.5)
        try:
            candidates_top = np.percentile(z_values[mask], 99.5)
        except IndexError:
            # Fails if there are no points in the masked array
            continue

        # Oude Elberink require the height difference > 0.1 m
        # And mark only the points 10 cm below the top as rail point candidates
        if candidates_top - ground_level > 0.1:
            mask = (z_values > candidates_top - 0.1) & (z_values < candidates_top + 0.05)

            # Also make sure these are only a minority of the points (otherwise it's a slope)
            if mask.sum() < majority_tresh * len(z_values):  # z.B. 0.3
                points['Classification'][indices[mask]] = RAIL


In [12]:
candidates = points[points["Classification"] == RAIL]
candidates.shape

(90623,)

## View Settings

In [13]:
# Viewsettings mit strg + c kopieren und hier einfügen

viewsettings = '''
{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 11.999975427985191, 11.99998692702502, 13.124079998226534 ],
			"boundingbox_min" : [ -13.000024572014809, -13.00001307297498, -3.9965200017734333 ],
			"field_of_view" : 60.0,
			"front" : [ -0.20468464372193082, -0.82045900926496551, 0.53380821531742795 ],
			"lookat" : [ -2.1145501200370735, -2.6052610037108783, 1.4494799802055294 ],
			"up" : [ 0.19010212482081987, 0.50164960558000959, 0.84392467398461002 ],
			"zoom" : 0.55999999999999983
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}

'''

viewsettings = json.loads(viewsettings)

front = viewsettings["trajectory"][0]["front"]
lookat = viewsettings["trajectory"][0]["lookat"]
up = viewsettings["trajectory"][0]["up"]
zoom = viewsettings["trajectory"][0]["zoom"]

## Kandidaten und Saatpunkte

Noise Filter auf Ergebnis des Voxelfilters

In [14]:
# filters.outlier sets Classification to 7, filters.range removes the points with Classification 7

noise_filter = pdal.Filter("filters.outlier", method="statistical", mean_k=10, multiplier=2.0).pipeline(candidates) | pdal.Filter("filters.range", limits="Classification![7:7]")
print(noise_filter.toJSON())
noise_filter.execute()
candidates = noise_filter.arrays[0]
candidates.shape 

[{"type": "filters.outlier", "method": "statistical", "mean_k": 10, "multiplier": 2.0, "tag": "filters_outlier1"}, {"type": "filters.range", "limits": "Classification![7:7]", "tag": "filters_range1"}]


(88499,)

In [15]:
xyz = np.vstack((candidates['X'], candidates['Y'], candidates['Z'])).transpose()
offset = xyz.mean(axis=0).round() 

xyz -= offset

In [16]:
pcd_candidates = o3d.geometry.PointCloud()
pcd_candidates.points = o3d.utility.Vector3dVector(xyz)
pcd_candidates.paint_uniform_color([0.5, 0.5, 0.5])

PointCloud with 88499 points.

Saatpunkte

In [17]:
low_intensity = candidates[candidates["Intensity"] < intensity_threshold]
low_intensity.shape

(63156,)

In [18]:
xyz_low = np.vstack((low_intensity['X'], low_intensity['Y'], low_intensity['Z'])).transpose()
xyz_low -= offset

pcd_low_intensity = o3d.geometry.PointCloud()
pcd_low_intensity.points = o3d.utility.Vector3dVector(xyz_low)
pcd_low_intensity.paint_uniform_color([0, 0, 0.7])

PointCloud with 63156 points.

In [19]:
# Downsample with poisson sampling

downsampling_pipeline = pdal.Filter("filters.sample", radius=downsample_radius).pipeline(low_intensity)
downsampling_pipeline.execute()
seed_points = downsampling_pipeline.arrays[0]
seed_points.shape 

(362,)

In [ ]:
xyz_seed = np.vstack((seed_points['X'], seed_points['Y'], seed_points['Z'])).transpose()
xyz_seed -= offset

seed_point_count = xyz_seed.shape[0]

pcd_seed_points = o3d.geometry.PointCloud()
pcd_seed_points.points = o3d.utility.Vector3dVector(xyz_seed)
pcd_seed_points.paint_uniform_color([1, 0, 0])

PointCloud with 362 points.

Erste Visualisierung: 
- blau: Kandidatenpunkte niedrige Intensität
- grau: Kandidatenpunkte hohe Intensität
- rot: Saatpunkte

In [21]:
o3d.visualization.draw_geometries([
    pcd_candidates, 
    pcd_low_intensity, 
    pcd_seed_points
    ], front=front, lookat=lookat, up=up, zoom=zoom)

Optionional nur Saatpunkte visualisieren

In [22]:
# o3d.visualization.draw_geometries([pcd_seed_points], front=front, lookat=lookat, up=up, zoom=zoom)

k-d-Baum

In [23]:
# k-D tree with all candidate points
tree = KDTree(xyz)  

In [24]:
# indices: ndarray (dtype object) with a list of indices for each seed point
indices = tree.query_ball_point(xyz_seed, r=neighborhood_radius)

In [32]:
def pca(cloud):
    """Use PCA to get einvalues and eigenvectors of a point cloud"""
    mean = np.mean(cloud, axis=0)
    centered = cloud - mean
    cov_matrix = np.cov(centered, rowvar=False) # row variance nicht berechnen
    eigenvals, eigenvecs = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvals)[::-1]
    sorted_eigenvals = eigenvals[sorted_indices]
    sorted_eigenvecs = eigenvecs[:,sorted_indices]
    # Returned vectors are in columns, first vector is eigenvec[:, 0] == eigenvec.T[0]
    return sorted_eigenvals, sorted_eigenvecs

def linearity(eigenvals):
    """Calculate the linearity of a point cloud"""
    return (eigenvals[0] - eigenvals[1]) / eigenvals[0]

## Visualisierung der linearity und Richtung um Saatpunkte 

In [ ]:
pcd_hood = pv.PolyData(xyz_seed)

In [ ]:
linearity_at_seed = np.empty((seed_point_count,1), dtype=float)
first_vec = np.zeros((seed_point_count,3), dtype=float)

In [ ]:
for i in range(seed_point_count):
    hood = xyz[indices[i]]
    if hood.shape[0] > 20:   
        eigenvals, eigenvects = pca(hood)
        linearity_at_seed[i] = linearity(eigenvals)
        first_vec[i] = eigenvects.T[0]
    else:
        linearity_at_seed[i] = np.nan

In [ ]:
pcd_hood['linearity'] = linearity_at_seed 

In der Visualisierung können mit dem Schieberegler alle Punkte mit geringerer lineary als der angezeigte Schwellenwert ausgeblendet werden. Dies erlaubt es, einen geeigneten Schwellenwert zu ermitteln. Die roten Linien zeigen die Richtung des ersten Eigenvektors.

Kononen et al. (2024) verwenden threshold 0.98 (behalten aber zusätzlich auch Punkte in Nachbarschaften mit hoher Punktdichte)

In [ ]:
p = pv.Plotter()


# Linien anzeigen (langsam!)
first_vec_scaled = 0.5 * first_vec
for i in range(len(xyz_seed)):
    start_point = xyz_seed[i]
    end_point = start_point + first_vec_scaled[i]
    line = pv.Line(start_point, end_point)
    p.add_mesh(line, color='red')


p.add_mesh_threshold(pcd_hood, 'linearity', title="Linearity", all_scalars=True, render_points_as_spheres=True, point_size=10)
p.show()

Widget(value='<iframe src="http://localhost:45155/index.html?ui=P_0x7f56f44a3070_3&reconnect=auto" class="pyvi…